In [1]:
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
CUSTOM_MODEL_NAME = 'my_ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
LABEL_MAP_NAME = 'label_map.pbtxt'

ROOT_WORKSPACE = 'workspace_tiffanies'

import os
paths = {
    'APIMODEL_PATH': 'object_detection',
    'WORKSPACE_PATH': ROOT_WORKSPACE,
    'ANNOTATION_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','data'),
    'CHECKPOINT_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model'),
    'OUTPUT_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model', 'export'), 
    'TFJS_PATH':os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME,'model_data','model','tfjsexport'), 
    'IMAGE_PATH': os.path.join(ROOT_WORKSPACE,'images'),
    'PRETRAINED_MODEL_PATH': os.path.join(ROOT_WORKSPACE,CUSTOM_MODEL_NAME),
    'SCRIPTS_PATH': 'scripts'
}

files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(os.path.join(paths['CHECKPOINT_PATH'], 'export','saved_model'))

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 8.197011947631836 seconds


In [3]:
from object_detection.utils import label_map_util

category_index = label_map_util.create_category_index_from_labelmap(os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt'),use_display_name=True)

print(category_index)

{1: {'id': 1, 'name': 'burj'}, 2: {'id': 2, 'name': 'empire'}}


In [6]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

cap = cv2.VideoCapture(2)

while True:
    # Read frame from camera
    ret, frame = cap.read()
    
    image_np = np.array(frame)
    
     # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))

    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.15,
        agnostic_mode=False)

    # Display output
    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

cap = cv2.VideoCapture(2)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height

while True:
    ret, frame = cap.read()
    image_np = np.array(frame)
   
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))

    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.15,
        agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows() 
cv2.waitKey(1) # normally unnecessary, but it fixes a bug on MacOS where the window doesn't close